<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C14_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
g=tf.Graph()


In [0]:
with g.as_default():
  t1=tf.constant(1)
  t2=tf.constant([1,2,3])
  t3=tf.constant([[[1],[2]],[[3],[4]]])
  
  r1=tf.rank(t1)
  r2=tf.rank(t2)
  r3=tf.rank(t3)
  
  s1=t1.get_shape()
  s2=t2.get_shape()
  s3=t3.get_shape()
  print("Shapes : ",s1,s2,s3)
  
with tf.Session(graph=g) as sess:
  print("Ranks : ",r1.eval(),r2.eval(),r3.eval())

Shapes :  () (3,) (2, 2, 1)
Ranks :  0 1 3


In [0]:
g1=tf.Graph()

In [0]:
with g1.as_default():
  a=tf.constant(5)
  b=tf.constant(2)
  c=tf.constant(10)
  z=2*(a-b)+c
with tf.Session(graph=g1) as sess:
  p=sess.run(z)
  print(p)

16


In [0]:
import tensorflow as tf
g1=tf.Graph()
with g1.as_default():
  tf_a=tf.placeholder(tf.int32,shape=[],name='tf_a')
  tf_b=tf.placeholder(tf.int32,shape=[],name='tf_b')
  tf_c=tf.placeholder(tf.int32,shape=[],name='tf_c')
  r1=tf_a-tf_b
  r2=2*r1
  z=r2+tf_c

In [0]:
  with tf.Session(graph=g1) as sess:
    feed={tf_a:1,
         tf_b:2,
         tf_c:3}
    print('z:',sess.run(z,feed_dict=feed))

z: 1


In [0]:
import tensorflow as tf
g2=tf.Graph()
with g2.as_default():
  tf_x=tf.placeholder(tf.float32,shape=[None,2],name='tf_x')
  x_mean=tf.reduce_mean(tf_x,axis=0,name='mean')
  

In [0]:
import numpy as np
np.random.seed(123)
np.set_printoptions(precision=2)
with tf.Session(graph=g2) as sess:
  x1=np.random.uniform(low=0,high=10,size=(5,2))
  print("Feeding Shape : ",x1.shape)
  print("Result : ",sess.run(x_mean,feed_dict={tf_x:x1}))
  x2=np.random.uniform(low=2,high=5,size=(10,2))
  print("Feeding Shape : ",x2.shape)
  print("Result : ",sess.run(x_mean,feed_dict={tf_x:x2}))

Feeding Shape :  (5, 2)
Result :  [6.21 4.68]
Feeding Shape :  (10, 2)
Resu;t :  [3.39 3.46]


In [0]:
import tensorflow as tf
import numpy as np
g3=tf.Graph()
with g3.as_default():
  w=tf.Variable(np.array([1,2]),name='weight')
  print(w)

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'weight:0' shape=(2,) dtype=int64_ref>


In [0]:
with tf.Session(graph=g3) as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(w))

[1 2]


In [0]:
import tensorflow as tf
g4=tf.Graph()
with g4.as_default():
  a=tf.Variable(1,name='a')
  init=tf.global_variables_initializer()
  b=tf.Variable(2,name='b')  

In [0]:
with tf.Session(graph=g4) as sess:
  sess.run(init)
  print('a : ',sess.run(a))

a :  1


In [0]:
with tf.Session(graph=g4) as sess:
  sess.run(init)
  print("b : ",sess.run(b))

b :  2


In [0]:
import tensorflow as tf
g5=tf.Graph()

with g5.as_default():
  with tf.variable_scope("net1"):
    with tf.variable_scope("layer1"):
      a1=tf.Variable(tf.random_normal(shape=(10,4)),name="a1")
    with tf.variable_scope("layer2"):
      a2=tf.Variable(tf.random_normal(shape=(5,3)),name='a2')
  with tf.variable_scope("net2"):
    a3=tf.Variable(2,name='a3')
  
  print(a1)
  print(a2)
  print(a3)
  

<tf.Variable 'net1/layer1/a1:0' shape=(10, 4) dtype=float32_ref>
<tf.Variable 'net1/layer2/a2:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'net2/a3:0' shape=() dtype=int32_ref>


In [0]:
import tensorflow as tf
import numpy as np


###----Helper Function----###


def build_classifier(data,labels,n_classes=2):
  data_shape=data.get_shape().as_list()
  weights=tf.get_variable(shape=(data_shape[1],n_classes),name='weights',dtype=tf.float32)
  bias=tf.get_variable(name='bias',initializer=tf.zeros(shape=n_classes))
  logits=tf.add(tf.matmul(data,weights),bias,name='logits')
  return logits,tf.nn.softmax(logits)
def build_generator(data,n_hidden):
  data_shape=data.get_shape().as_list()
  w1=tf.Variable(tf.random_normal(shape=(data_shape[1],n_hidden)),name='w1')
  b1=tf.Variable(tf.zeros(shape=n_hidden),name='b1')
  hidden=tf.add(tf.matmul(data,w1),b1,name='hidden_pre-activation')
  hidden=tf.nn.relu(hidden,'hidden_activation')
  
  w2=tf.Variable(tf.random_normal(shape=(n_hidden,data_shape[1])),name='w2')
  b2=tf.Variable(tf.zeros(shape=data_shape[1]),name='b2')
  output=tf.add(tf.matmul(hidden,w2),b2,name="output")
  return output,tf.nn.sigmoid(output)


In [0]:
###----Building Graph----###
batch_size=64
g5=tf.Graph()
with g5.as_default():
  tf_X=tf.placeholder(shape=(batch_size,100),dtype=tf.float32,name='tf_X')
  
  #---build the generator---#
  with tf.variable_scope('generator'):
    gen_out1=build_generator(data=tf_X,n_hidden=50)
  
  #---build the classifier---#
  with tf.variable_scope('classifier') as scope:
    #-Classify original data-#
    cls_out1=build_classifier(data=tf_X,labels=tf.ones(shape=batch_size))
    
    #-reuse classifier for generating data-#
    scope.reuse_variables()
    cls_out2=build_classifier(data=gen_out1[1],labels=tf.zeros(shape=batch_size))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
!rm -rf ./logs/ 

In [0]:
# Ensure TensorFlow 2.0 is installed.
# !pip install tensorflow==2.0.0-alpha0
# !pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard module is not an IPython extension.


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0-alpha0


In [0]:
with tf.Session(graph=g5) as sess:
  sess.run(tf.global_variables_initializer())
  file_writer=tf.summary.FileWriter(logdir='./log/',graph=g5)
  

In [0]:
%tensorboard --logdir 'log/'

UsageError: Line magic function `%tensorboard` not found.


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2019-04-24 11:05:38--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.237.103, 52.73.9.93, 52.4.95.48, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.237.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  18.4MB/s    in 0.8s    

2019-04-24 11:05:39 (18.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://238944b0.ngrok.io
